In [1]:
import numpy as np
import xmlTranslator as xmlT
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2
import importlib
from utils import get_project_root
from ipywidgets import FloatProgress

[1707995463.452241] [d72355bbe943:2185319:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


TODO: 
- Repeat the sequences 3 times
- Extend all chords into 18 enharmonic
- Complete the voicing as a MIDI array 
- Split into train and test
- Pass the MIDI into Embeddings

In [104]:
directory = '/data/iRealXML'
root = get_project_root()
myPath = str(root) + str(directory)

theChordDataset, theOffsetDataset, all_meta = xmlT.parse_info_from_XML(myPath)
print(theChordDataset.shape, theOffsetDataset.shape, len(all_meta))

  0%|          | 0/4007 [00:00<?, ?it/s]

(4006,) (4006,)
(4006,) (4006,) 4006


In [105]:
#find add 4 subtract 3 add 4 subtract 3
for song in theChordDataset:
    for i, e in enumerate(song):
        if 'alter #5 add 4 subtract 3' in e:
            print(song)
            print(i)
            break

['<style>' 'Ballad' '|:' 'Form_A' '.' 'G' 'm' '|' '.' 'Eb' 'maj7' '/' 'G'
 '|' '.' 'E' 'm7' 'alter b5' '|' '.' 'C#' 'm7' 'alter b5' '.' 'F#' '7' '|'
 '.' 'B' 'm7' '.' 'D' 'm7' '|' '.' 'A' '13' 'alter b9' '.' 'Bb' '/' 'A'
 '|' '.' 'D' 'm7' '|' '.' 'D' '7' 'add b9' '|' '.' 'C' 'm7' '.' 'G' 'm7'
 '|' '.' 'E' 'm7' 'alter b5 add b9' '.' 'E' '7' 'alter b5 add b9' '|' '.'
 'A' 'm' '|' '.' 'A' 'm' 'add #7' '|' '.' 'F' 'maj9' '.' 'C' 'maj7' '|'
 '.' 'C' 'm7' '|' '.' 'B' '9sus4'
 'add 4 subtract 3 add b9 add 4 subtract 3' '.' 'B' '7'
 'add 4 subtract 3 add b9 add 4 subtract 3' '|' '.' 'B' 'm7' '.' 'E' '7'
 'add b9' '|' 'Form_B' '.' 'F#' 'm7' '|' '.' 'C' '/' 'B' '.' 'F#' '7'
 'add b9' '|' '.' 'B' '9sus4' 'add 4 subtract 3 add b9 add 4 subtract 3'
 '.' 'B' '7' 'add 4 subtract 3 add b9 add 4 subtract 3' '/' 'B' '|' '.'
 'B' '7' 'alter #5' '.' 'B' '/' 'A' '|' '.' 'G' 'maj7' 'alter #5' '.' 'E'
 'm' '/' 'G' '|' '.' 'F#' 'm7' 'alter b5' '|' '.' 'C' 'maj7' '|' '.' 'B'
 'm7' '.' 'E' '7' 'add b9' '|' '.' 

In [106]:
importlib.reload(xmlT)

id = 0
expand_form_dataset = []
status = True
for i in tqdm(range(theChordDataset.shape[0])):
    song_structure, status = xmlT.expand_song_structure(theChordDataset[i], i)
    if status == False:
        print('Song:', all_meta[i])
        id = i
        break
    song_structure = np.array(song_structure)
    expand_form_dataset.append(song_structure)


 11%|█         | 433/4006 [00:00<00:00, 4300.88it/s]

100%|██████████| 4006/4006 [00:00<00:00, 4103.86it/s]


In [107]:
#check the length of the song

max = {'v': 0, 'id': 0}
min = {'v': 1000, 'id': 0}

for i in range(len(expand_form_dataset)):
    if len(expand_form_dataset[i]) > max['v']:
        max['v'] = len(expand_form_dataset[i])
        max['id'] = i
    if len(expand_form_dataset[i]) < min['v']:
        min['v'] = len(expand_form_dataset[i])
        min['id'] = i

print('Max:', max, 'Min:', min)


Max: {'v': 1321, 'id': 141} Min: {'v': 30, 'id': 492}


In [108]:
importlib.reload(xmlT)

#correct the wrong chords
corrected_sequence = xmlT.repleaceTheseChords(expand_form_dataset, False)

  0%|          | 0/4006 [00:00<?, ?it/s]

In [109]:
#Repeat the sequences 3 times
concatenated_array = []

concatenated_array = np.hstack(corrected_sequence) 

vocab = set(concatenated_array)

print(len(vocab))
print(vocab)

288
{'sus7', 'aug', 'alter b5', 'alter #11 add b9', '128 Feel', 'add #11 add b9 alter #5', 'Samba Reggae', 'Up Samba', 'add #9 add b9 alter b5', 'Medium Shuffle', 'add b9 add #11', 'o7 alter #7', 'aug7', 'add #7 add #11', 'add b13 add b9', 'alter #5 add 7', 'Samba Funk', 'dom7 add 9', 'UP Swing', 'Waltz', 'Rock Ballad', 'Gypsy Waltz', 'Blues Rock', 'Form_B', 'add 7 add b13', 'Funk', 'add b13 alter b5', 'G#', 'Form_Segno', 'm', 'Son', 'Ballad', 'Cb', 'Repeat_2', 'Worship', 'Bb', 'add 9 alter #5', 'Rock-Folk', 'Folk-Rock', 'Samba Enredo', 'add b6 add #9', 'alter #5', 'alter #11', 'sus7 add #11', 'Db', 'add 2 add #9', 'alter #5 add #11', 'C#', 'add #7 add b9', 'm6', 'Afoxé', 'Slow Shuffle', 'm add 6 add 9', 'add #9 alter #5', 'alter b5 add #11', 'Medium Swing', 'Reggae Pop', 'add b9 alter b5', '/', 'sus4', "Even 8th's", 'add #7 add 9', 'Form_C', 'Merengue', 'Up Tempo Swing', 'add #11 add b9', 'add #11 alter b5 add #9', 'Up Tempo', 'add 7 add 9', 'add #9 add b13', 'add 7 alter #5', 'Gypsy 

In [110]:
#Add the maj token to the chords
import voicing as vc
voicing = vc.Voicing()
#add the maj token to the chords
chord_progression = None
chord_progression = voicing.add_maj_token(corrected_sequence)

print(chord_progression[3751])


['<style>', 'Funk', '|', 'Form_intro', '.', 'Eb', 'maj7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', '|', '.', 'C', 'm7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', 'Form_intro', '.', 'Eb', 'maj7', '/', 'F', '|', '.', 'Bb', 'maj7', '/', 'F', '|', '.', 'C', 'm7', '/', 'F', '|', '.', 'D', 'm7', '.', 'G', 'dom7 add 9', '|', 'Form_A', '.', 'C', 'm7 add 9', '|', '.', 'F', 'sus7 add 9', '.', 'Eb', 'sus7 add 9', '|', '.', 'Bb', 'maj7', '/', 'D', '|', '.', 'G', 'sus7 add 9', '.', 'G', 'dom7 add 9', '|', '.', 'C', 'm7 add 9', '|', '.', 'F', 'sus7 add 9', '.', 'F', 'dom7 add 9', '|', '.', 'Db', 'maj', '/', 'C', '|', '.', 'Bb', 'sus7', 'add 7', '.', 'Bb', 'm', '.', 'Bb', 'dom7', '|', '.', 'Eb', 'm7 add 9', '|', '.', 'Ab', 'sus7 add 9', '.', 'Ab', 'm7 add 9', '|', '.', 'Db', 'maj7', '/', 'A', '|', '.', 'G', 'sus7 add 9', '.', 'G', 'dom7 add 9', '|', '.', 'C', 'sus7 add 9', '.', 'C', 'dom7 add 9', '|', '.', 'F', 'sus7 add 9', '.', 'F', 'dom7', '|', '.', 'Bb', 'maj7', '|', '.', 'G', 'sus7 add 9', 

In [111]:
#fix the format of extensions to avoid repeated elements
natures = {'maj', 'm', 'm6', 'm7', 'dom_7', 'maj7', 'o7', 'o', 'sus', 'sus2', 'sus7', 'ø7', 'power', 'm_maj7'}
id = 3751

for song in tqdm(chord_progression):
    for i, element in enumerate(song):
        if 'add' in element or 'alter' in element:
            #print(element)
            split = element.split(' ')
            size = len(split)
            if size > 2:
                if size%2 == 1:
                    #print('odd', split)
                    nature = split[0]
                    song[i] = nature
                    counter = 1 
                    for n in range(1, size-1, 2):
                        ext = split[n] + ' ' + split[n+1]
                        song.insert(i+counter, ext)
                        counter += 1
    #need to do it separately to avoid the index out of range           
    for i, element in enumerate(song):
        if 'add' in element or 'alter' in element:
            #print(element)
            split = element.split(' ')
            size = len(split)
            if size > 2:   
                if size%2 == 0:
                    #delete the element
                    song.pop(i)
                    counter = 0
                    for n in range(0, size-1, 2):
                        ext = split[n] + ' ' + split[n+1]
                        song.insert(i+counter, ext)
                        counter += 1


 52%|█████▏    | 2082/4006 [00:00<00:00, 10197.71it/s]

100%|██████████| 4006/4006 [00:00<00:00, 9973.73it/s] 


In [58]:
#find 
id = 0
find = ""

for i in range(len(chord_progression)):
    if find in chord_progression[i]:
        print(i)
        print(chord_progression[i])
        id = i
        #break
    
print(all_meta[id])


{'composer': 'George Gershwin, Ira Gershwin (1927)', 'style': 'Medium Swing', 'song_name': "'S Wonderful", 'tonality': 'Eb major', 'midi_key': 15, 'time_signature': '4/4', 'decade': 'Null'}


In [112]:
#find duplicated 'add' or 'alter' elements and remove them

sub_group = chord_progression[0:160]
structural_elements = {'.', '|', ':|', '|:', '/', 'N.C.'}
dataset_duplicates = chord_progression

for x, song in tqdm(enumerate(dataset_duplicates)):
    for y, e in enumerate(song):
        if '.' in e:
            search = True
            stepper = y + 2
            check_extensions = []
            while search:
                if (stepper) >= len(song):
                    search = False
                    #print('End of song')
                    break
                elif stepper < len(song) - 1:
                    element = song[stepper]
        
                if element in structural_elements or element.startswith('Form_'):
                    search = False
                else:
                    if element not in check_extensions:
                        check_extensions.append(element)
                    elif stepper < len(song)-1:
                        #print('Duplicated:', x, stepper, element, check_extensions)
                        song.pop(stepper)
                    stepper += 1
            if len(check_extensions) > 2:
                continue
                print(x, y+stepper, check_extensions)
                

1530it [00:00, 7545.72it/s]

4006it [00:00, 7231.37it/s]


In [115]:
#fix this ones
'''
['dom7', 'sus7', 'add b9'] remove dom7
['sus7', 'add 13', 'sus4'] remove sus4
['sus7', 'add 9', 'sus4'] remove sus4
['m', 'add 6', 'add 9', 'sus4'] remove sus4
['sus7', 'add 13', 'sus4', 'add b9'] remove sus4
['dom7', 'add 13', 'sus4', 'add b9'] remove sus4
'''
fix_this = [{'seq': ['dom7', 'sus7', 'add b9'], 'remove': 'dom7'},
            {'seq': ['sus7', 'add 13', 'sus4'], 'remove': 'sus4'},
            {'seq': ['sus7', 'add 9', 'sus4'], 'remove': 'sus4'},
            {'seq': ['m', 'add 6', 'add 9', 'sus4'], 'remove': 'sus4'},
            {'seq': ['sus7', 'add 13', 'sus4', 'add b9'], 'remove': 'sus4'},
            {'seq': ['dom7', 'add 13', 'sus4', 'add b9'], 'remove': 'sus4'}]

structural_elements = {'.', '|', ':|', '|:', '/', 'N.C.'}

for x, song in enumerate(dataset_duplicates):
    for y, e in enumerate(song):
        if '.' in e:
            search = True
            stepper = y + 2
            check_extensions = []
            while search:
                if (stepper) >= len(song):
                    search = False
                    #print('End of song')
                    break
                elif stepper < len(song) - 1:
                    element = song[stepper]
        
                if element in structural_elements or element.startswith('Form_'):
                    search = False
                else:
                    if element not in check_extensions:
                        check_extensions.append(element)
                    for item in fix_this:
                        if check_extensions == item['seq']:
                            print(x, stepper, check_extensions, 'to remove:', item['remove'])
                            #find the location of the element in the array
                            for i in range(len(check_extensions)):
                                if check_extensions[i] == item['remove']:
                                    location = i + stepper - 2
                                    #erase this element from song
                                    if location < len(song):
                                        print(location, item['remove'], song[location])
                                        song.pop(location)
                                    break
                            
                    stepper += 1


In [116]:
id = 1269
print(all_meta[id])
print(chord_progression[id])


{'composer': 'Bernard Ighner', 'style': 'Ballad', 'song_name': 'Everything Must Change', 'tonality': 'F minor', 'midi_key': 17, 'time_signature': '4/4', 'decade': 'Null'}
['<style>', 'Ballad', '|', 'Form_A', '.', 'F', 'm7', 'add 9', '|', '.', 'C', 'dom7', '/', 'E', '|', '.', 'F', 'm7', 'add 9', '/', 'E', '|', '.', 'D', 'dom7', 'alter b5', '|', '.', 'Db', 'maj7', '.', 'Ab', 'maj', '/', 'C', '|', '.', 'Bb', 'm7', '.', 'Bb', 'm7', '/', 'A', '|', '.', 'G', 'm7', 'alter b5', '|', '.', 'C', 'dom7', 'add #9', 'alter #5', '|', '.', 'F', 'm7', 'add 9', '|', '.', 'C', 'dom7', '/', 'E', '|', '.', 'F', 'm7', 'add 9', '/', 'E', '|', '.', 'D', 'dom7', 'add #9', 'alter b5', '|', '.', 'Db', 'maj7', 'add 9', '.', 'Ab', 'maj', '/', 'C', '|', '.', 'Bb', 'm7', '.', 'Bb', 'm7', '/', 'A', '|', '.', 'Gb', 'maj7', 'add 9', 'add #11', '|', '.', 'Eb', 'sus7', 'add 13', 'add b9', '.', 'Eb', 'dom7', 'add 13', 'sus4', '|', '.', 'Ab', 'maj7', 'add 9', '|', '.', 'G', 'm7', 'alter b5', 'add b9', 'alter #5', '.', 'C',